# Questions on Catch the Phase.



## P.1



1. State the motivation behind the quantum phase estimation (QPE) subroutine. \
Answer: QPE is used to solve linear system of equations and finding the ground energy if a molecule. 

2. List some applications of QPE. \
Answer: Mentioned in 1. 

3. Describe the phase kickback technique. \
Answer: Let us assume we have an controlled unitary operator $U$ and its corresponding eigenval and eigen vec $\lambda$ and $\ket{\psi_{\lambda}}$. We can prepare the target wires in $U$ eigen-vector states. The effect of applying $U$ on $\ket{\psi_\lambda}$ is defined as 
$$
U \ket{\psi_\lambda} = \lambda \ket{\psi_\lambda} = e^{2i \pi \theta } \ket{\psi_\lambda}
$$ 
the eigenvalue corresponding to this eigenstate is kicked back to the control wires. Then, the control wires pick up a phase of $e^{2i \pi \theta }$. This is called phase kickback.

4. Estimate a phase using phase kickback. \
Answer: The phase via kickback is $e^{2i \pi \theta }$.

In [28]:
import pennylane as qml
from pennylane import numpy as np

In [29]:
Z_gate = np.asarray([[1,1],[1,-1]])
eigen_value_z,eigen_vec_z = np.linalg.eig(Z_gate)
print(f"The Eigen values are: {eigen_value_z}")
print(f"The eigen vectors are:{eigen_vec_z}")

The Eigen values are: [ 1.41421356 -1.41421356]
The eigen vectors are:[[ 0.92387953 -0.38268343]
 [ 0.38268343  0.92387953]]


In [30]:
lamda_1_z,lamda_2_z = eigen_value_z
psi_1_z,psi_2_z = eigen_vec_z

Since the eigenvectors form an orthonormal set, all unitary matrices can be written in a diagonal representation or an orthonormal decomposition (or equivalently, a spectral decomposition).
$
\begin{equation}
U = \sum_{\lambda} e^{2i \pi \theta} \ket{\psi_{\lambda}} \bra{\psi_{\lambda}}
\end{equation}
$
where $e^{2i \pi \theta}$ is an eigenvalue of $\ket{ \psi_{\lambda} }$

In [31]:
U = 0
for i in range(len(eigen_value_z)):
    U += eigen_value_z[i]*np.kron(eigen_vec_z[i],eigen_vec_z[i])
print(f"The unitary matrix is: {U}")


The unitary matrix is: [ 1. -1. -1. -1.]


Exercise P.1.3. Write the Tgate and the X gate in terms of their eigenbases.

In [32]:
X_gate = np.array([[0,1],[1,0]])
T_gate = np.asarray([[1,0],[0,np.exp(1j*np.pi/4)]])
eigen_val_x,eigen_vec_x = np.linalg.eigh(X_gate)
eigen_val_t,eigen_vec_t = np.linalg.eigh(T_gate)
print(f"The Eigen values of X gate are: {eigen_val_x}")
print(f"The eigen vectors of X gate are:{eigen_vec_x}")
print()
print(f"The Eigen values of T gate are: {eigen_val_t}")
print(f"The eigen vectors of T gate are:{eigen_vec_t}")

The Eigen values of X gate are: [-1.  1.]
The eigen vectors of X gate are:[[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]

The Eigen values of T gate are: [0.70710678 1.        ]
The eigen vectors of T gate are:[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [33]:
U_x,U_t = 0,0
for i in range(len(eigen_val_x)):
    U_x += eigen_val_x[i]*np.kron(eigen_vec_x[i],eigen_vec_x[i])
    U_t += eigen_val_t[i]*np.kron(eigen_vec_t[i],eigen_vec_t[i])

print(f"The unitary matrix of X gate is: {U_x}")
print()
print(f"The unitary matrix of T gate is: {U_t}")

The unitary matrix of X gate is: [0. 1. 1. 0.]

The unitary matrix of T gate is: [1.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [34]:
CNOT_gate = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
Toffoli_gate = np.identity(8)
Toffoli_gate[-1,-1],Toffoli_gate[-2,-2], = 0,0
Toffoli_gate[-1,-2],Toffoli_gate[-2,-1], = 1,1
eigen_val_cnot,eigen_vec_cnot = np.linalg.eigh(CNOT_gate)
eigen_val_toff,eigen_vec_toff = np.linalg.eigh(Toffoli_gate)

prove that the probability of observing 0 when measuring the first qubit is ($1 - sin^2 \pi \theta$) and  and the probability of observing 1 is $sin^2 \pi \theta$ \
The circuit is $H-CU-H (\ket{0} \otimes \ket{\psi})$ where $\ket{\psi}$ is an eigen vector if controlled unitary operator $CU$. 

Applying the given gate in orders, we will be at state,
$$
\ket{\Phi} = \frac{1}{2}[(1 + e^{2 i \pi \theta}) \ket{0}  + (1 - e^{2 i \pi \theta}) \ket{1} ] \otimes \ket{\psi} \\

Therefore, P(\ket{0}) = \frac{(1 + e^{2 i \pi \theta})}{2^2}^2 = 1-sin^2 \pi \theta \\

and, P(\ket{1}) = sin^2 \pi \theta
$$

### Codercise P.1.1.
 You are given a unitary that is promised to be either the Z gate or the −Z gate. Write a quantum program using phase kickback that will result in the state |0⟩ with a probability of 100% on the first qubit if Z is applied and |1⟩ with a probability of 100% on the first qubit if −Z is applied.

In [35]:
dev = qml.device("default.qubit", wires=2, shots=1)

@qml.qnode(dev)
def guess_the_unitary(unitary):
    """Given a unitary that performs a Z or a -Z operation
    on a qubit, guess which one it is.
    
    Args: 
        U (array[complex]): A unitary matrix, guaranteed to be either Z or -Z.
    
    Returns:
        array [int]:  Probabilities on  on the first qubit
        using qml.probs()
    """
    ##################
    # YOUR CODE HERE #
    ##################  
    qml.Hadamard(wires = 0)
    qml.ControlledQubitUnitary(unitary, control_wires = 0, wires = 1)
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [0])
    

# Z gate 
U = qml.PauliZ.matrix 

# -Z gate
# U = (-1)*qml.PauliZ.matrix

# print(guess_the_unitary(U))


### Codercise P.1.2. 
Find the eigenvalues of the X gate

In [36]:
dev = qml.device("default.qubit", wires=2)
        
@qml.qnode(dev)
def phase_kickback_X(eigenvector):
    """ Given an eigenvector of X, 
    apply the phase kickback circuit to observe 
    the probabilities on the control wire
    
    Args: 
        eigenvector(String): A string "plus" or "minus" depicting 
        the eigenvector of X
    
    Returns:
        array[int]: Measurement outcome on the first qubit using qml.probs()
    """
    # Prepare |ψ>
    ##################
    # YOUR CODE HERE #
    ##################  
    if eigenvector == "minus":  
        qml.PauliX (wires = 1)
    qml.Hadamard(wires = 1)
    qml.Hadamard(wires = 0)
    qml.CNOT(wires = [0,1])
    qml.Hadamard(wires = 0)

    # Phase kickback
    ##################
    # YOUR CODE HERE #
    ################## 
 
    return qml.probs(wires=[0])   

print(phase_kickback_X("plus"))
print(phase_kickback_X("minus"))

# MODIFY EIGENVALUES BELOW 
eigenvalue_of_X_plus = 1
eigenvalue_of_X_minus = -1


[1. 0.]
[0. 1.]


## P.2

### Codercise P.2.1. 
Given a unitary matrix U , compute the value of a higher power, $U^{2^k}$. You can use the matrix_power function from NumPy's linear algebra library.

In [37]:
def U_power_2k(unitary, k):
    """ Computes U at a power of 2k (U^2^k)
    
    Args: 
        unitary (array[complex]): A unitary matrix
    
    Returns: 
        array[complex]: the unitary raised to the power of 2^k
    """
    ##################
    # YOUR CODE HERE #
    ##################
    pwr = np.power(2,k)
    return np.linalg.matrix_power(unitary,pwr )
            

# Try out a higher power of U
U = qml.T
print(U)

U_power_2k(T_gate, 2)
    
   


<class 'pennylane.ops.qubit.non_parametric_ops.T'>


tensor([[ 1.+0.00000000e+00j,  0.+0.00000000e+00j],
        [ 0.+0.00000000e+00j, -1.+3.99346924e-16j]], requires_grad=True)

### Codercise P.2.2. 
Implement a subroutine that applies the sequence of $U^{2^k}$ unitaries on the target wires controlled on the estimation wires. The function U_power_2k from the previous exercise is available for you to use.

In [38]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def apply_controlled_powers_of_U(unitary):
    """A quantum function that applies the sequence of powers of U^2^k to
    the estimation wires.
    
    Args: 
        unitary (array [complex]): A unitary matrix
    """

    ##################
    # YOUR CODE HERE #
    ##################
    i = 0
    for wire in reversed(range(len(estimation_wires))):
        u_mat = U_power_2k(unitary,wire)
        qml.ControlledQubitUnitary(u_mat,control_wires = estimation_wires[i], wires = target_wires)
        i += 1
    
apply_controlled_powers_of_U(T_gate)

### Codercise P.2.3. 
Implement the QPE subroutine given a unitary, a set of estimation wires, and a set of target wires. The functions defined above (U_power_2k and apply_controlled_powers_of_U ) are available to use. Additionally, the function prepare_eigenvector which prepares an eigenvector of the unitary operator is also given to you below. To implement the $QFT^{\dagger}$
, you can make use of PennyLane's template for QFT and qml.adjoint.

In [39]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """ Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Measurement outcome probabilities on the estimation wires.
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    for wire in estimation_wires:
        qml.Hadamard(wires = wire)
    prepare_eigenvector()
    apply_controlled_powers_of_U(unitary)
    qml.adjoint(qml.QFT)(wires = estimation_wires)
    return qml.probs(wires = estimation_wires)
    
    

# U = qml.T._matrix()
print(qpe(T_gate))


[8.54017922e-33 1.00000000e+00 2.83952386e-32 2.43148655e-32
 3.85054815e-32 5.56593754e-32 9.51759284e-32 2.40741243e-31]


### Codercise P.2.4. 
Given the probabilities on the estimation wires, estimate the phase associated with the T gate, when the eigenvector is prepared in the state
$\ket{1}$.

In [40]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def estimate_phase(probs):
    """Estimate the value of a phase given measurement outcome probabilities
    of the QPE routine.
    
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
    
    Returns:
        float: the estimated phase   
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    temp = 1/np.power(2,len(estimation_wires))
    list_ = [temp*i for i in range(len(probs))]
    return float(np.sum(np.array(probs)*list_))

# U = qml.T._matrix()

# probs = qpe(U)
probs = [1.3479e-32,1.0000e+00, 2.7843e-32 ,2.4652e-32 ,3.3868e-32 ,5.6093e-32, 8.4698e-32 ,2.6193e-31]


estimated_phase = estimate_phase(probs)
print(estimated_phase)

0.125


### Codercise P.2.5.
Use PennyLane's template for QPE to calculate the phase of the T gate.

In [41]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Probabilities on the estimation wires.
    """
    
    prepare_eigenvector()
    
    ##################
    # YOUR CODE HERE #
    ################## 
    qml.QuantumPhaseEstimation(unitary, target_wires, estimation_wires)
    return qml.probs(wires = estimation_wires)


# U = qml.T._matrix()
# probs = qpe(U)
print(estimate_phase(probs))


0.125


## P.3

### Let's be rational (Theory)

What if there is no $t-bits$ binary expansion for the phase?

During our $QPE$ when we apply the $\textbf{ORACLE}$ $ H^{\otimes n}$ on circuit, we arrive at $QFT$ base. Recall QFT is defined as,
$$
U_{QFT} \ket{x} = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{\frac{2i\pi}{N} x . y} \ket{y}, \text{where } N = 2^n
$$ 

What if phase can not be represented by bilary representation, i.e something like $\theta = 0.\theta_1 \theta_2 \theta_1 \theta_2 ... \rightarrow \theta =  0.\overline{\theta_1 \theta_2}$. Then we need to approx the t-bits probability. If $\tilde{\theta}$ is the real phase, we can define $\tilde{\theta}  = \theta + \delta$ where $\delta = \frac {1}{2^t}$ and apply $QFT^{\dagger}$ on the above state. Doing so, we will get the success probability as, 
$$
 \left |\frac{1}{2^t} \sum_{y=0}^{2^t -1 }e ^{\frac{2 i \pi \delta y}{2^t}} \right |^2
$$
This is similar to geometric series. So the success prob can be solved to ,
$$
P = \frac{1}{2^t} \frac{1 - e^{2 i \pi \delta}}{1 - e^{\frac{2i\pi \delta} {2^t}}}
$$
Solving it, we get,
$$
 \left |\frac{1}{2^t} \sum_{y=0}^{2^t -1 }e ^{\frac{2 i \pi \delta y}{2^t}} \right |^2 = \left | \left ( \frac{2}{\pi}\right )  \right |^2 = 0.4053
$$


### Codercise

### Codercise P.3.1. 
Using the QPE algorithm, estimate the two most likely outcomes of the eigenphase of the eigenvector $\ket{1}$
 of a unitary operator. The function `qpe(unitary, estimation_wires, target_wires)` is provided for you and returns a list of probabilities on the estimation wires. Since this involves testing your solution for a different number of estimation wires, please change the return value of the variable done to True when you are finished with testing.

In [42]:
dev = qml.device("default.qubit", wires=10)

def fractional_binary_to_decimal(binary_fraction, wires):
    return float(binary_fraction/ 2 ** len(wires))

def phase_window(probs, estimation_wires):
    """ Given an array of probabilities, return the phase window of the unitary's eigenvalue
    
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
        estimation_wires (list[int]): List of estimation wires
    
    Returns:
        (float, float): the lower and upper bound of the phase
    """
    index_1 = np.argmax(probs)
    bound_1 = fractional_binary_to_decimal(index_1,estimation_wires)
    probs[index_1] = 0.0
    index_1 = np.argmax(probs)
    bound_2 = fractional_binary_to_decimal(index_1,estimation_wires)
    ##################
    # YOUR CODE HERE #
    ################## 
    # bound_1 = 0 # MOST LIKELY OUTCOME
    #bound_2 = 1 #SECOND MOST LIKELY OUTCOME
    return (bound_2, bound_1)


# Test your solution

# You can increase the number of estimation wires to a maximum of range(0, 9)
estimation_wires = range(0, 3)

# The target is set to the last qubit
target_wires = [9]

# Define the unitary
#U = np.array([[1, 0], [0, np.exp((2*np.pi*1j/7))]])

#probs = qpe(U, estimation_wires, target_wires)

print(phase_window(probs, estimation_wires))

# MODIFY TO TRUE AFTER TESTING YOUR SOLUTION
done = True


(0.875, 0.125)


### Codercise P.3.2.
 On a device with 10 qubits, the last wire is set as the target wire and the first 9 wires are set as the estimation wires. Calculate the phase estimates starting at 1 to a maximum of 9 estimation wires. The qpe(unitary, estimation_wires, target_wires) and phase_window(probs, estimation_wires) functions have been defined for you and the target_wires have been prepared in the state $\ket{1}$. Hit the submit button to see how the phase window gets smaller (and the estimate gets closer to the actual phase) as the number of estimation wires increases for the following unitary:
 $ \begin{bmatrix}
1 & 0 \\
0 & e^{\frac{2i\pi}{7}}
\end{bmatrix} $

In [43]:
dev = qml.device("default.qubit", wires=10)

def estimates_array(unitary):
    """ Given a unitary, return a list of its phase windows
    
    Args: 
        unitary (array[complex]): A unitary matrix.
    
    Returns:
        [(float, float)]: a list of phase windows for 1 to 9 
        estimation wires
    """

    estimates = []
    target_wires = [9]

    ##################
    # YOUR CODE HERE #
    ################## 
    for i in range(2,10):
        probs = qpe(unitary, range(i), target_wires)
        bounds = phase_window(probs, range(i))
        estimates.append(bounds)
        

    return estimates

# Define the unitary
U = np.array([[1, 0], [0, np.exp((2*np.pi*1j/7))]])

#estimates_array(U)

###################
# SUBMIT FOR PLOT #
###################
